This notebook is for building AutoLinearizers.

In [1]:
import json

import video2vision as v2v
import numpy as np
from PIL import Image, ImageDraw

from crosshairs import SampleSelector
from utils import get_size

IMAGE_PATH = '/data/hot/color/camera_data/2022-09-23/VIS_vid/C0002.MP4'
EXPECTED_VALUES_PATH = '../data/aruco_samples.csv'
SAMPLE_REFLECTIVITIES_PATH = '../data/aruco_samples.csv'
CAMERA_SENSE_PATH = '../data/camera_sensitivities.csv'
OUTPUT_PATH = 'autolinearizer.json'

We begin by running the ARUCO detector on the input image.

In [2]:
image_size = get_size(IMAGE_PATH)
reader = v2v.Loader(IMAGE_PATH, image_size)

for t, (frame, _) in enumerate(reader):
    try:
        _, corners = v2v.utils.locate_aruco_markers(frame, [0, 1, 2, 3])
    except RuntimeError:
        continue
    else:
        break
else:
    raise RuntimeError('Failed to locate markers')

TypeError: 'NoneType' object is not subscriptable

We outline the ARUCO markers in green.

In [ ]:
corners = corners.reshape(4, 4, 2).astype(np.int64)
image = Image.fromarray(np.clip(256 * frame, 0, 255).astype(np.uint8))
draw = ImageDraw.Draw(image)

for box in corners:
    for (x1, y1), (x2, y2) in zip(box[:-1], box[1:]):
        draw.line((x1, y1, x2, y2), fill=(0, 255, 0), width=4)
    draw.line((*box[-1], *box[0]), fill=(0, 255, 0), width=4)

image = np.array(image)

Please then click on the centers of the samples.

In [ ]:
selector = SampleSelector(
    # Loader returns in BGR, but SampleSelector expects RGB
    image[:, :, ::-1],
    (1000, 1000),
    scale_crosshair=True,
    include_numbers=True,
)
display(selector)

Finally, we load and calculate the expected values from the reflectivities and camera sensitivities, and build and save the AutoLinearizer.

In [ ]:
linearization_sample_ref = np.genfromtxt(SAMPLE_REFLECTIVITIES_PATH, skip_header=True, delimiter=',')[:, 1:]
camera_sense = np.genfromtxt(CAMERA_SENSE_PATH, skip_header=True, delimiter=',')[:, 1:]
# Assume isoluminance
pred_qc = linearization_sample_ref.T.dot(camera_sense)

In [ ]:
autoline_op = v2v.AutoLinearize(
    marker_ids=[0, 1, 2, 3],
    marker_points=corners,
    sample_points=selector.crosshairs,
    expected_values=pred_qc,
)

In [ ]:
with open(OUTPUT_PATH, 'w') as out_file:
    json.dump(autoline_op._to_json(), out_file)